In [0]:
import pandas as pd
import time
import re
import numpy as np

In [0]:
data_path = 'C:/Users/User/Desktop/NUS/DSA4261/data/'

def get_data(data_path,year,month,zone):
    data = pd.read_csv(data_path + "ais_southern_ships_{}_{}_zone{}_reduce.csv".format(str(year),str(month),str(zone)))
    return data

def get_yearly_data(data_path,year,zone):
    df = pd.DataFrame()
    for i in range(1,3): #load 2 months (can be changed)
        data = get_data(data_path,str(year),"%02d" % i,str(zone))
        df = pd.concat([df,data],axis=0)
    return df

**Data Cleaning**

In [0]:
df = get_yearly_data(data_path,2017,15)
df = df.dropna(subset=['LAT','LON']).reset_index(drop=True)

from datetime import datetime
basedatetime_utc = df['BaseDateTime'].apply(lambda x: datetime.strptime(x,'%Y-%m-%dT%H:%M:%S.0000000Z'))
basedatetime_unix = df['BaseDateTime'].apply(lambda x: datetime.strptime(x,'%Y-%m-%dT%H:%M:%S.0000000Z').timestamp())
date = df['BaseDateTime'].apply(lambda x:x[:10])

df['BaseDateTime_UTC'] = basedatetime_utc
df['BaseDateTime_UNIX'] = basedatetime_unix
df['MMSI'] = df['MMSI'].astype(str)
df['Date'] = date

df = df.sort_values(by=['MMSI','BaseDateTime_UNIX'])
df = df.reset_index(drop=True)
df = df[~df.Status.isin(['at anchor','moored'])] #Remove at anchor or moored ships as they are less likely to be bunkering

**Mapping VesselTypeCode to VesselType?** *Fishing, Tanker, Tug Tow, Cargo, etc..*

In [0]:
vessel_info_df = pd.read_csv('C:/Users/User/Desktop/NUS/DSA4261/VesselTypeCodes.csv')
vessel_info_dict = {}
for i in range(0,len(vessel_info_df)):
    vesselcode = float(vessel_info_df['VesselType'][i])
    vesselgroup = vessel_info_df['VesselGroup'][i]
    vessel_info_dict[vesselcode] = vesselgroup
df['VesselType_New'] = df['VesselType'].map(vessel_info_dict)

In [0]:
df.VesselType_New.value_counts()

**Function to get TimeDiff between current and previous point**

In [0]:
from datetime import datetime, timedelta

def time_diff_in_seconds(t1,t2):
    if t1 > t2:
        diff = (t1 - t2)
    elif t2 > t1:
        diff = (t2 - t1)
    elif t2 == t1:
        diff = 0
    return diff

In [0]:
mmsis = df['MMSI'].unique().tolist()
groupings = df.groupby('MMSI')
new_df = pd.DataFrame()
for mm in mmsis:
    group = groupings.get_group(mm).reset_index(drop=True).copy()
    group['TimeDiff'] = np.zeros(len(group))
    for j in range(1,len(group)):
        t1 = group['BaseDateTime_UNIX'][j]
        t2 = group['BaseDateTime_UNIX'][j-1]
        td = time_diff_in_seconds(t1,t2)
        group['TimeDiff'][j] = td/group['BaseDateTime_UNIX'].sum()     #TimeDiff Ratio 
    new_df = pd.concat([new_df,group],axis=0)

In [0]:
df = new_df

**Convert datapoints to Trajectories**

In [0]:
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from fiona.crs import from_epsg
import movingpandas as mpd

In [0]:
geometry = [Point(xy) for xy in zip(df['LON'],df['LAT'])]
crs = from_epsg(4326) 
df['t'] = pd.to_datetime(df['BaseDateTime'],format='%Y-%m-%dT%H:%M:%S.0000000Z')
df = df.set_index('t')
df= df.rename(columns={"LAT":"Latitude","LON":"Longitude"})
geo_df = GeoDataFrame(df, crs=crs)
geo_df['geometry'] = geometry

In [0]:
MIN_LENGTH = 2 # meters
traj_collection = mpd.TrajectoryCollection(geo_df, 'MMSI', min_length=MIN_LENGTH)
print("Finished creating {} trajectories".format(len(traj_collection)))

**Extract Lat, Lon, Sog, TimeDiff as features to be put into a distance matrix computation**

In [0]:
trajectories = traj_collection.trajectories
traj_len = len(trajectories)
trajectories_list = [trajectories[i].df[['Latitude','Longitude','SOG','TimeDiff']].reset_index(drop=True).to_numpy() for i in range(traj_len)]

In [0]:
import matplotlib.pyplot as plt
def plot_trajectories(traj_lst):
    for traj in traj_lst:
        plt.plot(traj[:,0], traj[:,1])
    plt.show()

In [0]:
plot_trajectories(trajectories_list)

**Hausdorff Distance is used as the distance metric in the distance matrix computation**

In [0]:
from scipy.spatial.distance import directed_hausdorff
from sklearn.cluster import DBSCAN

def hausdorff( u, v):
    d = max(directed_hausdorff(u, v)[0], directed_hausdorff(v, u)[0])
    return d

traj_count = len(trajectories_list)
D = np.zeros((traj_count, traj_count))

# Distance Matrix computation.. This may take a while...
for i in range(traj_count):
    for j in range(i + 1, traj_count):
        distance = hausdorff(trajectories_list[i], trajectories_list[j])
        D[i, j] = distance
        D[j, i] = distance

In [0]:
D.shape 

In [0]:
D

**DBSCAN** 

In [0]:
kms_per_radian = 6371.0088
epsilon = 10000 / kms_per_radian

mdl = DBSCAN(eps=epsilon, min_samples=2,metric="precomputed",n_jobs=-1)
cluster_lst = mdl.fit_predict(D) #Where D is the distance matrix computed via Hausdorff Distance

In [0]:
cluster_lst

In [0]:
# Append cluster results to main dataframe

mapping  = {}
for i in range(len(cluster_lst)):
    mapping[i] = cluster_lst[i]
    
df_full = pd.DataFrame()
for i in range(len(trajectories)):
    df_current = trajectories[i].df
    df_current['cluster'] = mapping[i]
    df_full = pd.concat([df_full,df_current],axis=0)

In [0]:
#df_full.to_csv('full_dbscan.csv',index=False)  #Save results for further analysis in QGIS

**Some other useful functions**

In [0]:
# Extract Tankers' trajectory index and MMSI from the collection of trajectories(trajectories)

tankers_idx = []
for i in range(traj_len):
    if trajectories[i].df['VesselType_New'][0] == 'Tanker':
        print(i, trajectories[i].df['MMSI'][0])
        tankers_idx.append(i)

In [0]:
# Get nearest neighbours of tanker with tanker_idx based on Distance Matrix, D

def get_nn(D,tanker_idx,listtankers=tankers_idx):
    current_min = 200000000
    current_idx = -1
    idxs = []
    for i in range(len(D[:,tanker_idx])):
        if i not in listtankers:
            d = D[:,tanker_idx][i]
            idxs.append([d,i])
    idxs.sort(key=lambda x:x[0])
    return idxs

In [0]:
get_nn(D,31)  #For instance, if a tanker has tanker_idx of 31 in the collection of trajectories(trajectories),
              #this function gives a list of trajectories that are closest to it in terms of the distance value
              #sorted in ascending order. 